In [1]:
import requests
r = requests.get('https://eangus.org/conference-attendance/')
print(r.text[0:500])

<!DOCTYPE html>
<html lang="en-US">
<head>
	<meta charset="UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<link rel="pingback" href="https://eangus.org/xmlrpc.php" />

	<script type="text/javascript">
		document.documentElement.className = 'js';
	</script>

	<script>var et_site_url='https://eangus.org';var et_post_id='156';function et_core_page_resource_fallback(a,b){"undefined"===typeof b&&(b=a.sheet.cssRules&&0===a.sheet.cssRules.length);b&&(a.onerror=null,a.onload=null,a.hre


In [2]:
#Ok, we have successfully scraped the html.  Let's turn it into a beautiful soup.
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
#ok, now I need to look into the html source code to see how I can use soup.find_all() here.  
#I'm guessing what I want is the class gv-field-20-1.  Let's check.
results = soup.find_all('span', attrs =  {'class':'gv-field-20-1'})
len(results)

0

In [5]:
#that doesn't appear to have worked. what about
results = soup.find_all('td', attrs = {'class':'gv-field-20-1'})
len(results)

25

In [6]:
#ah, good, this works then. let's see.
results[0:3]

[<td class="gv-field-20-1" id="gv-field-20-1">Bonnie</td>,
 <td class="gv-field-20-1" id="gv-field-20-1">CHRIS</td>,
 <td class="gv-field-20-1" id="gv-field-20-1">DONNIE</td>]

In [8]:
#Ah, so I've just pulled the first names.  Hmmm...ah, they actually labelled their attributes.  so gv-field-20-1 is first
#names.  So I should look into the html where these classes are defined.  Ah, the defining attributes are right above the body.
#what if I just use gv-field-20* or something?
#Let's brute force it first, then figure out something more eloquent afterwards.
first_name = soup.find_all('td', attrs = {'class':'gv-field-20-1'})
len(first_name)


25

In [9]:
#Nope, doesn't work that way.  what about 'span' 'gv-field-label'?
results = soup.find_all('span', attrs = {'class': 'gv-field-label'})
len(results)

12

In [10]:
#how odd.  What does it look like?
results[0:11]

[<span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=1&amp;dir=asc"></a> First Name</span>,
 <span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=3&amp;dir=asc"></a> Last Name</span>,
 <span class="gv-field-label"><a class="gv-sort gv-icon-sort-desc" href="/conference-attendance/?sort=4&amp;dir=desc"></a> Association</span>,
 <span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=7&amp;dir=asc"></a> Ticket Type</span>,
 <span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=8&amp;dir=asc"></a> Attendee Status</span>,
 <span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=9&amp;dir=asc"></a> Meal</span>,
 <span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=1&amp;dir=asc"></a> First N

In [19]:
#ah, exactly as one would expect, it labels the column/field. So we have:
#gv-field-20-1 first name
#gv-field-20-3 last name
#gv-field-20-4 Association
#gv-field-20-7 Ticket Type
#gv-field-20-8 Attendee Status
#gv-field-20-9 Meal
last_name = soup.find_all('td', attrs = {'class':'gv-field-20-3'})
association = soup.find_all('td', attrs = {'class':'gv-field-20-4'})
ticket_type = soup.find_all('td', attrs = {'class':'gv-field-20-7'})
status = soup.find_all('td', attrs = {'class':'gv-field-20-8'})
meal = soup.find_all('td', attrs = {'class':'gv-field-20-9'})
all_attributes = [first_name, last_name, association, ticket_type, status, meal]


In [15]:
for attribute in all_attributes:
    print(len(attribute))

25
25
25
25
25
25


In [20]:
#alright, let's see if I can paste this together like I want.
#actually, let's see if I need to clean up the presentation at all.
last_name[0:3]


[<td class="gv-field-20-3" id="gv-field-20-3">Moser</td>,
 <td class="gv-field-20-3" id="gv-field-20-3">GRAGG</td>,
 <td class="gv-field-20-3" id="gv-field-20-3">KYLE</td>]

In [44]:
#all right, I need to rip the text out of this. how about using class 'alt'?
results_alt = soup.find_all('tr', attrs = {'class':'alt'}) 
len(results_alt)

13

In [48]:
results_blank = soup.find_all('tr', attrs = {'class': ""})
len(results_blank)

14

In [47]:
print(results_alt)
#we can see results_alt is names all the way down, so I"m picking up extra data with the "" bit.

[<tr class="alt">
<td class="gv-field-20-1" id="gv-field-20-1">Bonnie</td><td class="gv-field-20-3" id="gv-field-20-3">Moser</td><td class="gv-field-20-4" id="gv-field-20-4"></td><td class="gv-field-20-7" id="gv-field-20-7">2015 Attendee Ticket (Regular)</td><td class="gv-field-20-8" id="gv-field-20-8">Attendee</td><td class="gv-field-20-9" id="gv-field-20-9">Beef</td> </tr>, <tr class="alt">
<td class="gv-field-20-1" id="gv-field-20-1">DONNIE</td><td class="gv-field-20-3" id="gv-field-20-3">KYLE</td><td class="gv-field-20-4" id="gv-field-20-4"></td><td class="gv-field-20-7" id="gv-field-20-7">2014 Pre-Registrations *CLOSED*</td><td class="gv-field-20-8" id="gv-field-20-8">Attendee</td><td class="gv-field-20-9" id="gv-field-20-9">No, I will not attend</td> </tr>, <tr class="alt">
<td class="gv-field-20-1" id="gv-field-20-1">Tonny</td><td class="gv-field-20-3" id="gv-field-20-3">Pridgen</td><td class="gv-field-20-4" id="gv-field-20-4">Alabama</td><td class="gv-field-20-7" id="gv-field-2

In [50]:
results_blank[0]

<tr>
<th class="gv-field-20-1" id="gv-field-20-1"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=1&amp;dir=asc"></a> First Name</span></th><th class="gv-field-20-3" id="gv-field-20-3"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=3&amp;dir=asc"></a> Last Name</span></th><th class="gv-field-20-4" id="gv-field-20-4"><span class="gv-field-label"><a class="gv-sort gv-icon-sort-desc" href="/conference-attendance/?sort=4&amp;dir=desc"></a> Association</span></th><th class="gv-field-20-7" id="gv-field-20-7"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=7&amp;dir=asc"></a> Ticket Type</span></th><th class="gv-field-20-8" id="gv-field-20-8"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=8&amp;dir=asc"></a> Attendee Status</span></th><th class="gv-field-20-9" id="gv

In [52]:
results_blank[-1]

<tr>
<th class="gv-field-20-1" id="gv-field-20-1"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=1&amp;dir=asc"></a> First Name</span></th><th class="gv-field-20-3" id="gv-field-20-3"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=3&amp;dir=asc"></a> Last Name</span></th><th class="gv-field-20-4" id="gv-field-20-4"><span class="gv-field-label"><a class="gv-sort gv-icon-sort-desc" href="/conference-attendance/?sort=4&amp;dir=desc"></a> Association</span></th><th class="gv-field-20-7" id="gv-field-20-7"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=7&amp;dir=asc"></a> Ticket Type</span></th><th class="gv-field-20-8" id="gv-field-20-8"><span class="gv-field-label"><a class="gv-sort gv-icon-caret-up-down" href="/conference-attendance/?sort=8&amp;dir=asc"></a> Attendee Status</span></th><th class="gv-field-20-9" id="gv

In [53]:
results_blank[1:-1]

[<tr class="">
 <td class="gv-field-20-1" id="gv-field-20-1">CHRIS</td><td class="gv-field-20-3" id="gv-field-20-3">GRAGG</td><td class="gv-field-20-4" id="gv-field-20-4"></td><td class="gv-field-20-7" id="gv-field-20-7">2014 Pre-Registrations *CLOSED*</td><td class="gv-field-20-8" id="gv-field-20-8">Attendee</td><td class="gv-field-20-9" id="gv-field-20-9">Beef</td> </tr>,
 <tr class="">
 <td class="gv-field-20-1" id="gv-field-20-1">Mike</td><td class="gv-field-20-3" id="gv-field-20-3">Schultz</td><td class="gv-field-20-4" id="gv-field-20-4"></td><td class="gv-field-20-7" id="gv-field-20-7">2015 INVITED DG-VIP</td><td class="gv-field-20-8" id="gv-field-20-8">Delegate (authorized and confirmed by your State Association)</td><td class="gv-field-20-9" id="gv-field-20-9">No, I will not attend</td> </tr>,
 <tr class="">
 <td class="gv-field-20-1" id="gv-field-20-1">Janine</td><td class="gv-field-20-3" id="gv-field-20-3">Pridgen</td><td class="gv-field-20-4" id="gv-field-20-4">Alabama</td><

In [54]:
len(results_blank[1:-1])

12

In [55]:
#that fixed it
page_results = results_alt + results_blank[1:-1]
len(page_results)

25

In [56]:
page_results[0]

<tr class="alt">
<td class="gv-field-20-1" id="gv-field-20-1">Bonnie</td><td class="gv-field-20-3" id="gv-field-20-3">Moser</td><td class="gv-field-20-4" id="gv-field-20-4"></td><td class="gv-field-20-7" id="gv-field-20-7">2015 Attendee Ticket (Regular)</td><td class="gv-field-20-8" id="gv-field-20-8">Attendee</td><td class="gv-field-20-9" id="gv-field-20-9">Beef</td> </tr>

In [58]:
#ok, we should be able to start cleaning things up now.
type(page_results[0])

bs4.element.Tag

In [60]:
first_name = page_results[0].find('gv-field-20-1')
print(first_name)

None


In [61]:
#bitch. what about contents?
page_results[0].contents[1]

<td class="gv-field-20-1" id="gv-field-20-1">Bonnie</td>

In [62]:
page_results[0].contents[1].text

'Bonnie'

In [67]:
#ok, so we'll need first_name = result.contents.text?  we shall see. let's test out a for loop now actually
for result in page_results:
    print(result.contents[1].text)

Bonnie
DONNIE
Tonny
Kristi
James (Jim)
JUDY
NANCY
BEVERLY
MARJORIE
AUX 4
STEVE
SHERMAN
JOHN
CHRIS
Mike
Janine
Pam
Pat
DON
EDDIE
CECIL
AUX 3
JUDY
PATTI
JACQUELYN


In [ ]:
#fuck yes. so the code will be first_name = result.contents[1].text

In [68]:
for result in page_results:
    print(result.contents[2].text)
    #so last_name = result.contents[2].text

Moser
KYLE
Pridgen
Taylor
Choate
WOOD
WALLER
PIKE
MONK
.
BOATWRIGHT
HARGROVE
ELMORE
GRAGG
Schultz
Pridgen
Adams
Choate
WOOD
PIKE
MONK
.
HOLLAND
REESOR
HARGROVE


In [72]:
page_results[3].contents[3]

<td class="gv-field-20-4" id="gv-field-20-4">Alabama</td>

In [75]:
for result in page_results:
    print(result.contents[3].text)
    # association = result.contents[3].text



Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama


Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama
Alabama


In [77]:
page_results[0].contents[4]

<td class="gv-field-20-7" id="gv-field-20-7">2015 Attendee Ticket (Regular)</td>

In [78]:
for result in page_results:
    print(result.contents[4].text)
    #ticket_type = result.contents[4].text

2015 Attendee Ticket (Regular)
2014 Pre-Registrations *CLOSED*
2015 Attendee Ticket (Regular)
2015 Attendee Ticket (Regular)
2015 Attendee Ticket (Regular)
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2015 INVITED DG-VIP
2015 Attendee Ticket (Regular)
2015 Attendee Ticket (Regular)
2015 Attendee Ticket (Regular)
2015 Executive Council/Conference Staff/National Office
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*
2014 Pre-Registrations *CLOSED*


In [79]:
page_results[0].contents[5]

<td class="gv-field-20-8" id="gv-field-20-8">Attendee</td>

In [80]:
for result in page_results:
    print(result.contents[5].text)
    #status = result.contents[5].text

Attendee
Attendee
Attendee
Delegate (authorized and confirmed by your State Association)
Attendee
Auxiliary
Attendee
Auxiliary
Attendee
Attendee
Attendee
Attendee
Attendee
Attendee
Delegate (authorized and confirmed by your State Association)
Attendee
Delegate (authorized and confirmed by your State Association)
Guest (of an Attendee)
Executive Council
Attendee
Attendee
Attendee
Attendee
Attendee
Attendee


In [85]:
page_results[2].contents[6]

<td class="gv-field-20-9" id="gv-field-20-9">Beef</td>

In [86]:
for result in page_results:
    print(result.contents[6].text)

Beef
No, I will not attend
Beef
Chicken
Chicken
Chicken
Beef
Beef
Chicken
Chicken
Beef
No, I will not attend
Beef
Beef
No, I will not attend
Chicken
Chicken
Beef
Beef
Beef
Beef
Chicken
No, I will not attend
Beef
Beef


In [87]:
records = []
for result in page_results:
    first_name = result.contents[1].text
    last_name = result.contents[2].text
    association = result.contents[3].text
    ticket_type = result.contents[4].text
    status = result.contents[5].text
    meal = result.contents[6].text
    records.append([first_name, last_name, association, ticket_type, status, meal])
    
records[0:3]

[['Bonnie', 'Moser', '', '2015 Attendee Ticket (Regular)', 'Attendee', 'Beef'],
 ['DONNIE',
  'KYLE',
  '',
  '2014 Pre-Registrations *CLOSED*',
  'Attendee',
  'No, I will not attend'],
 ['Tonny',
  'Pridgen',
  'Alabama',
  '2015 Attendee Ticket (Regular)',
  'Attendee',
  'Beef']]

In [90]:
import pandas as pd
df = pd.DataFrame(records, columns = ['First Name', 'Last Name', 'Association', 'Ticket Type', 'Attendee Status', 'Meal'])
df.head

<bound method NDFrame.head of      First Name   Last Name Association  \
0        Bonnie       Moser               
1        DONNIE        KYLE               
2         Tonny     Pridgen     Alabama   
3        Kristi      Taylor     Alabama   
4   James (Jim)      Choate     Alabama   
5          JUDY        WOOD     Alabama   
6         NANCY      WALLER     Alabama   
7       BEVERLY        PIKE     Alabama   
8      MARJORIE        MONK     Alabama   
9         AUX 4           .     Alabama   
10        STEVE  BOATWRIGHT     Alabama   
11      SHERMAN    HARGROVE     Alabama   
12         JOHN      ELMORE     Alabama   
13        CHRIS       GRAGG               
14         Mike     Schultz               
15       Janine     Pridgen     Alabama   
16          Pam       Adams     Alabama   
17          Pat      Choate     Alabama   
18          DON        WOOD     Alabama   
19        EDDIE        PIKE     Alabama   
20        CECIL        MONK     Alabama   
21        AUX 3         

Ok, this is a good spot to stop for the night.  Next I need to actually scroll through every page, or find some other solution. It would also not hurt to clean up some of these options for a cleaner data file.  Such as just 'Delegate' and No -> N/A, etc.